In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
%load_ext version_information
%version_information numpy, dlib, matplotlib, imageio, gaze, torch, torchvision

Software,Version
Python,3.6.9 64bit [GCC 8.4.0]
IPython,7.8.0
OS,Linux 4.15.0 112 generic x86_64 with Ubuntu 18.04 bionic
numpy,1.17.2
dlib,19.19.0
matplotlib,3.1.1
imageio,2.6.1
gaze,The 'gaze' distribution was not found and is required by the application
torch,1.5.1
torchvision,0.6.1


## skim through images which were wrongly classified

Some images were obviously misclassified at the supervision level, let's try to fix that.

In [4]:
import os
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive
import pickle
import imageio

In [5]:
def view_image(filename, target):
    
    img_face = imageio.imread(filename)
    fig, ax = plt.subplots(1, 1, figsize=(15, 8))
    ax.imshow(img_face)
    ax.set_title(target)
    #plt.show()
    
def change(filename, filename_face, target, new_target) :
    filename_new = filename.replace(target, new_target)
    filename_face_new = filename_face.replace(target, new_target)

    print('Changing...', filename, ' to ', filename_new)
    import shutil
    shutil.move(filename, filename_new)
    shutil.move(filename_face, filename_face_new)
    
    list_change.append([filename, filename_new])

In [6]:
def f(i, action, target):
    
    file = list_data[i]
    filename = dataset + '/' + target + '/' + file
    if not os.path.exists(filename) :
        for t in list_target :
            if os.path.exists(dataset + '/' + t + '/' + file) :
                filename = dataset + '/' + t + '/' + file
                target = t
    filename_face = filename.replace(dataset, dataset_face)
    print(filename)
    view_image(filename, target)
    
    if   action == 'center' : change(filename, filename_face, target, 'center')
    elif action == 'right'  : change(filename, filename_face, target, 'right')
    elif action == 'left'   : change(filename, filename_face, target, 'left')
    elif action == 'blink'  : change(filename, filename_face, target, 'blink')
    
    return action

In [7]:
with open('list_change.pkl', 'rb') as fichier :
    list_change = pickle.load(fichier, encoding='latin1')

In [8]:
len(list_change)

652

In [9]:
dataset_valid = '../dataset_valid'
dataset_new   = '../dataset_new'
dataset_face  = 'dataset_faces_20200731'


#-------------------------------------------------
list_target = ['center', 'right', 'left', 'blink']
target = 'blink'
dataset = dataset_valid
list_data = np.sort(os.listdir(dataset + '/' + target))
#-------------------------------------------------

In [10]:
len(list_data)

707

In [11]:
a = widgets.FloatText()
b = widgets.IntSlider(min=0, max=len(list_data)-1, step=1, value=0, readout=True)
mylink = widgets.jslink((a, 'value'), (b, 'value'))

w = interactive(f, i=b,
                action=widgets.ToggleButtons(options=['Keep']+list_target),
                target=target)

display(a, w)

FloatText(value=0.0)

interactive(children=(IntSlider(value=0, description='i', max=706), ToggleButtons(description='action', option…

In [12]:
len(list_change)

652

In [13]:
with open('list_change.pkl', 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(list_change)

In [14]:
nb = 0
for x in range(len(list_change)) :
    for y in range(x+1, len(list_change)) :
        if list_change[x][0]==list_change[y][1] and list_change[x][1]==list_change[y][0] :
            print(x, list_change[x], '\n', y, list_change[y], '\n')
            list_change[x][1] = list_change[y][1]
            list_change[y][0] = list_change[x][0]
            print(x, list_change[x], '\n', y, list_change[y], '\n\n')
            nb+=2
print(nb)

0


In [15]:
x, nb = 0, 0
while x < len(list_change) :
    if list_change[x][0]==list_change[x][1] :
        nb+=1
        print(nb, list_change[x])
        list_change.pop(x)
        x-=1
    x+=1     

In [16]:
print(len(list_change))

652


In [17]:
with open('list_change.pkl', 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(list_change)